
# Hollow Core in infinite glass 

N0 Configuration in Bird

In [ ]:
import numpy as np

from fiberamp.fiber.microstruct.bragg import BraggExact
from step_exact import plotlogf
from ngsolve.webgui import Draw
from ngsolve import CF
from scipy.optimize import newton


In [ ]:
n_air = 1.00027717
n_glass = 1.4388164768221814

n_high, n_low = 1.44, 1.43

In [ ]:
A = BraggExact(ts=[15e-6,15e-6], mats=['air', 'glass'], 
               ns=[lambda x:n_air, lambda x:n_glass],
              maxhs=[.2,.02], wl=1.8e-6, scale=1.5e-5)

# A = BraggExact(ts=[15e-6,15e-6], mats=['air', 'glass'], 
#                ns=[lambda x:n_high, lambda x:n_low],
#               maxhs=[.2,.05], wl=3.5e-6, scale=1.5e-5)

In [ ]:
Draw(CF([n for n in A.ns]), A.mesh)

In [ ]:
# k_low = A.k0 * A.ns[0] * A.scale
k_low, k_high = A.k0 * np.min(A.ns) * A.scale, A.k0 * np.max(A.ns) * A.scale

outer = 'h2'
Ktype = 'kappa'
nu = 1

In [ ]:
%matplotlib inline
plotlogf(A.determinant, k_low-.1, k_high+.1, -.1,.1, nu, outer,
         iref=100, rref=100, levels=100)

In [ ]:
%matplotlib notebook
plotlogf(A.determinant, .99*k_low, 1.0001*k_low, -.3,.3, nu, outer,
         iref=100, rref=100, levels=100, figsize=(12,8))

In [ ]:
guess = np.array(.999*k_low)

beta1 = newton(A.determinant, guess, args=(nu, outer, Ktype), tol = 1e-16)

print("Scaled beta: ", beta1, ". Residual of determinant: ", abs(A.determinant(beta1, nu, outer, Ktype)))


# Bragg Exact Fields

In [ ]:
Fs = A.all_fields(beta1, nu=nu, outer=outer, Ktype=Ktype)

In [ ]:
Draw(Fs['Ez'], A.mesh)

In [ ]:
Draw(Fs['Hz'], A.mesh)

In [ ]:
Draw(Fs['Etv'].real, A.mesh, vectors={'grid_size':150})


In [ ]:
Draw(Fs['Ephi'], A.mesh)

In [ ]:
Draw(Fs['Hphi'], A.mesh)

# Using StepExact for comparison

In [ ]:
import numpy as np

from step_exact import StepExact,plotlogf
from ngsolve.webgui import Draw
from ngsolve import CF
from scipy.optimize import newton
from step_exact.param_dicts.n0_params import params

In [ ]:
B = StepExact(params)

In [ ]:
Draw(CF([B.n_core,B.n_clad]), B.mesh)

In [ ]:
k_low = B.k0 * B.n_core * B.scale
k_high = B.k0 * B.n_clad * B.scale
clad = 'h2'
nu = 1
k_low, k_high

In [ ]:
plotlogf(B.hybrid_det, .99*k_low, 1.0001*k_low, -.1,.1, nu, clad,
         iref=100, rref=100, levels=100)

In [ ]:
plotlogf(B.hybrid_det, .995*k_low, k_low, -.025,.025, nu, clad,
         iref=100, rref=100, levels=100)

In [ ]:
guess = np.array(52.325)

beta2 = newton(B.hybrid_det, guess, args=(nu, clad), tol = 1e-15)

print("Scaled beta: ", beta2, ". Residual of determinant: ", abs(B.hybrid_det(beta2, nu, clad)))


In [ ]:
beta1, beta2

# Step Exact Fields

In [ ]:
Fs = B.hybrid_fields(beta1, nu, clad=clad)

In [ ]:
Draw(Fs['Ez'], B.mesh)

In [ ]:
Draw(Fs['Hz'], B.mesh)

In [ ]:
Draw(Fs['Ephi'], B.mesh)

In [ ]:
Draw(Fs['Hphi'], B.mesh)

# Notes:

- It appears that the determinant function is correct for BraggExact in the two layer case: both Step Exact and BraggExact find the same beta.  But I'm not displaying my fields correctly for BraggExact, so the place to look is in the creation of the coefficents and the fields in that class for errors.

- StepExact's eigenfunction has poles where BraggExact does not.  Also appears to have some continuum eigenvalues.  Not yet sure why these don't appear in BraggExact